# Introduction to Smolagents: Building a Bridge Reference Agent

Welcome to this hands-on introduction to the **smolagents** framework! In this notebook, you'll learn how to create AI agents that can search for information about bridges, preparing you for building more complex bridge design agents.

## What You'll Learn
1. Understanding AI agents and the smolagents framework
2. The difference between CodeAgent and ToolCallingAgent
3. Using built-in tools like web search
4. Creating custom tools
5. Building your own Bridge Reference Agent
6. Integrating agents into larger systems

Let's start by installing the necessary packages:

In [ ]:
# Install smolagents with toolkit support for web search
!pip install smolagents[toolkit] -q

## Part 1: Introduction to Smolagents

### What are AI Agents?

AI agents are autonomous systems that can:
- Understand tasks given in natural language
- Break down complex problems into steps
- Use tools to gather information or perform actions
- Reason about the results and provide answers

### The Smolagents Framework

Smolagents is a lightweight framework by Hugging Face that makes it easy to build AI agents. It provides:
- Simple decorators to create tools
- Two types of agents with different capabilities
- Built-in tools for common tasks
- Easy integration with various language models

In [ ]:
# Import the essentials
from smolagents import CodeAgent, ToolCallingAgent, InferenceClientModel, tool

# Initialize a language model (using Hugging Face's inference API)
# You'll need to set your HF_TOKEN environment variable or pass it here
model = InferenceClientModel()

print("Smolagents is ready to use!")

### Two Types of Agents

Smolagents offers two agent types, each with different strengths:

#### 1. **CodeAgent**
- Writes and executes Python code to solve tasks
- Can perform complex calculations and data manipulation
- More flexible but requires careful prompt engineering
- Outputs look like: `result = search_tool("query")`

#### 2. **ToolCallingAgent**
- Makes structured JSON tool calls
- More predictable and easier to control
- Better for production systems
- Outputs look like: `{"tool_call": {"name": "search_tool", "arguments": {"query": "..."}}}`

Let's see both in action:

In [ ]:
# Example 1: CodeAgent - writes Python code
code_agent = CodeAgent(tools=[], model=model)
result = code_agent.run("Calculate the golden ratio (1 + sqrt(5)) / 2")
print(f"CodeAgent result: {result}")

print("\n" + "="*50 + "\n")

# Example 2: ToolCallingAgent - makes structured calls
# (This will fail without tools, but shows the attempt)
tool_agent = ToolCallingAgent(tools=[], model=model)
try:
    result = tool_agent.run("What is the golden ratio?")
except Exception as e:
    print(f"ToolCallingAgent needs tools to work: {e}")

## Part 2: Using Built-in Tools

Smolagents comes with several built-in tools. The most useful for our bridge reference agent is the **WebSearchTool**, which uses DuckDuckGo to search the internet.

### Direct Tool Usage

Before giving tools to agents, let's understand how they work by using them directly:

In [ ]:
from smolagents import WebSearchTool

# Create an instance of the web search tool
search_tool = WebSearchTool()

# Use it directly to search for bridge information
results = search_tool("Golden Gate Bridge specifications length")
print("Search results:")
print(results[:500] + "..." if len(results) > 500 else results)

### Giving Tools to Agents

Now let's give the search tool to our agents and see how they use it differently:

In [ ]:
# CodeAgent with web search
code_agent_with_search = CodeAgent(
    tools=[search_tool], 
    model=model,
    verbosity_level=2  # Show what the agent is doing
)

print("=== CodeAgent with WebSearchTool ===")
result = code_agent_with_search.run(
    "Find the length of the Golden Gate Bridge"
)
print(f"\nFinal answer: {result}")

In [ ]:
# ToolCallingAgent with web search
tool_agent_with_search = ToolCallingAgent(
    tools=[search_tool], 
    model=model,
    verbosity_level=2
)

print("=== ToolCallingAgent with WebSearchTool ===")
result = tool_agent_with_search.run(
    "Find the length of the Golden Gate Bridge"
)
print(f"\nFinal answer: {result}")

## Part 3: Creating Custom Bridge Tools

While WebSearchTool is great, we can create specialized tools for our bridge domain. Let's learn how to create tools using the `@tool` decorator.

### Anatomy of a Tool

Every tool needs:
1. A **name** - unique identifier
2. A **description** - tells the agent when to use it
3. **Input parameters** with type hints
4. An **output type**
5. A docstring with an **Args:** section

In [ ]:
# Create a specialized bridge information tool
@tool
def search_bridge_info(bridge_name: str) -> str:
    """
    Search for detailed information about a specific bridge including 
    its specifications, history, and engineering details.
    
    Args:
        bridge_name: The name of the bridge to search for
    """
    # We'll use the WebSearchTool internally but add bridge-specific context
    search = WebSearchTool()
    query = f"{bridge_name} bridge specifications history engineering length height type material"
    return search(query)

# Test the tool directly
result = search_bridge_info("Brooklyn Bridge")
print(result[:500] + "...")

In [ ]:
# Create a tool to find bridges by type
@tool
def find_bridges_by_type(bridge_type: str, region: str = "worldwide") -> str:
    """
    Find examples of bridges of a specific type in a given region.
    
    Args:
        bridge_type: Type of bridge (e.g., suspension, arch, beam, cable-stayed, truss)
        region: Geographic region to search in (default: worldwide)
    """
    search = WebSearchTool()
    query = f"famous {bridge_type} bridges {region} examples list"
    return search(query)

# Test finding suspension bridges
suspension_bridges = find_bridges_by_type("suspension", "USA")
print("Suspension bridges in USA:")
print(suspension_bridges[:500] + "...")

In [ ]:
# Create a tool to compare bridge spans
@tool
def compare_bridge_spans(bridge1: str, bridge2: str) -> str:
    """
    Compare the main span lengths of two bridges.
    
    Args:
        bridge1: Name of the first bridge
        bridge2: Name of the second bridge
    """
    search = WebSearchTool()
    # Search for both bridges
    info1 = search(f"{bridge1} bridge main span length meters")
    info2 = search(f"{bridge2} bridge main span length meters")
    
    return f"Information about {bridge1}:\n{info1[:300]}\n\nInformation about {bridge2}:\n{info2[:300]}"

# Test comparison
comparison = compare_bridge_spans("Golden Gate Bridge", "Brooklyn Bridge")
print(comparison)

## Part 4: Building the Bridge Reference Agent

Now let's combine our custom tools into a proper Bridge Reference Agent. We'll create both CodeAgent and ToolCallingAgent versions to see the differences.

In [ ]:
# Collect all our bridge tools
bridge_tools = [
    search_bridge_info,
    find_bridges_by_type,
    compare_bridge_spans,
    WebSearchTool()  # Keep general search as fallback
]

# Create a CodeAgent version
bridge_agent_code = CodeAgent(
    tools=bridge_tools,
    model=model,
    verbosity_level=1,
    max_steps=5
)

print("Bridge Reference Agent (CodeAgent) is ready!")

In [ ]:
# Test the CodeAgent with a complex query
result = bridge_agent_code.run(
    "Find me three examples of suspension bridges built after 2000 and tell me their main span lengths"
)
print("\n=== FINAL RESULT ===")
print(result)

In [ ]:
# Create a ToolCallingAgent version
bridge_agent_tool = ToolCallingAgent(
    tools=bridge_tools,
    model=model,
    verbosity_level=1,
    max_steps=5
)

# Test with the same query
result = bridge_agent_tool.run(
    "What type of bridge is the Tower Bridge in London and when was it built?"
)
print("\n=== FINAL RESULT ===")
print(result)

### Comparing Agent Behaviors

Let's see how each agent type handles the same task differently:

In [ ]:
# Simple comparison task
task = "Which has a longer span: Golden Gate Bridge or Akashi Kaikyo Bridge?"

print("=== CodeAgent Approach ===")
code_result = bridge_agent_code.run(task)
print(f"Result: {code_result}")

print("\n" + "="*50 + "\n")

print("=== ToolCallingAgent Approach ===")
tool_result = bridge_agent_tool.run(task)
print(f"Result: {tool_result}")

## Part 5: Integration with the Triage System

In a real bridge design system, our Bridge Reference Agent would work alongside other specialized agents. The **managed agents** pattern allows a manager agent to delegate tasks to specialized agents.

Here's how to prepare our agent for integration:

In [ ]:
# Create a specialized bridge reference agent with name and description
# This makes it ready for use as a managed agent

bridge_reference_agent = ToolCallingAgent(
    tools=bridge_tools,
    model=model,
    name="bridge_reference_agent",
    description="Searches for information about existing bridges, their types, specifications, and engineering details. Use this when you need reference information about real bridges."
)

# Create a simple manager agent that can delegate to our bridge agent
manager = CodeAgent(
    tools=[],  # Manager doesn't need direct tools
    model=model,
    managed_agents=[bridge_reference_agent],
    verbosity_level=2
)

print("Manager agent with bridge reference capability is ready!")

In [ ]:
# Test delegation - the manager will automatically use the bridge agent
result = manager.run(
    "I'm designing a new suspension bridge. Can you find me information about "
    "the top 3 longest suspension bridges in the world and their main specifications?"
)

print("\n=== FINAL RESULT ===")
print(result)

### How This Fits Into the Bridge Design System

In the full system, you have:
1. **Triage Agent** (manager) - Coordinates all tasks
2. **Geometry Agent** - Creates 3D models in Rhino/Grasshopper
3. **Bridge Reference Agent** - Provides real-world examples (what we built!)
4. *Future: Safety Agent* - Checks safety parameters
5. *Future: Materials Agent* - Manages material selection

The triage agent decides which specialist to consult based on the task.

In [ ]:
# Example of how it would work in the full system
# (This is pseudocode for illustration)

example_system_prompt = """
# In the actual bridge design system:

user_request = "Design a suspension bridge similar to the Golden Gate Bridge but for a 500m span"

# Triage agent would:
1. Ask bridge_reference_agent: "Get specifications of Golden Gate Bridge"
2. Ask geometry_agent: "Create bridge with 500m span based on these specs: ..."
3. Coordinate between agents to refine the design
"""

print(example_system_prompt)

## Part 6: Hands-on Exercises

Now it's your turn! Try these exercises to deepen your understanding.

### Exercise 1: Bridge Materials Tool

Create a tool that searches for information about materials used in different bridge types.

In [ ]:
# Exercise 1: Create a bridge materials tool
# TODO: Create a tool called 'search_bridge_materials' that:
# - Takes a bridge_type parameter (e.g., "suspension", "arch")
# - Optionally takes a specific material to focus on
# - Returns information about materials commonly used in that bridge type

# Your code here:



# Test your tool:
# result = search_bridge_materials("cable-stayed")
# print(result[:500])

In [ ]:
# Solution (hidden - try yourself first!)
"""
@tool
def search_bridge_materials(bridge_type: str, material_focus: str = "") -> str:
    '''
    Search for information about materials used in specific bridge types.
    
    Args:
        bridge_type: Type of bridge (suspension, arch, beam, etc.)
        material_focus: Optional specific material to focus on (steel, concrete, etc.)
    '''
    search = WebSearchTool()
    
    if material_focus:
        query = f"{bridge_type} bridge {material_focus} material properties usage"
    else:
        query = f"{bridge_type} bridge construction materials steel concrete composite"
    
    return search(query)

# Test
result = search_bridge_materials("suspension", "steel cables")
print(result[:500])
"""

### Exercise 2: Bridge History Timeline

Create an agent that can find and organize historical information about bridges in chronological order.

In [ ]:
# Exercise 2: Create a bridge history agent
# TODO: Create an agent that can:
# - Search for historical information about multiple bridges
# - Organize the information chronologically
# - Present it in a timeline format

# Hint: You might want to create a new tool for searching bridge history
# and then use a CodeAgent to process and organize the results

# Your code here:



# Test your agent:
# result = bridge_history_agent.run("Create a timeline of famous suspension bridges built in the 20th century")
# print(result)

In [ ]:
# Solution (hidden - try yourself first!)
"""
@tool
def search_bridge_history(bridge_name: str) -> str:
    '''
    Search for historical information about a bridge including construction dates.
    
    Args:
        bridge_name: Name of the bridge
    '''
    search = WebSearchTool()
    query = f"{bridge_name} bridge construction date year built opened history timeline"
    return search(query)

# Create an agent that can organize information chronologically
bridge_history_agent = CodeAgent(
    tools=[search_bridge_history, find_bridges_by_type],
    model=model,
    additional_authorized_imports=['datetime', 're'],
    verbosity_level=1
)

# Test with a timeline request
result = bridge_history_agent.run(
    'Find information about 3 famous suspension bridges and organize them by construction date'
)
print(result)
"""

## Summary and Next Steps

Congratulations! You've learned:
- ✅ The difference between CodeAgent and ToolCallingAgent
- ✅ How to use built-in tools like WebSearchTool
- ✅ How to create custom tools with the @tool decorator
- ✅ How to build agents that can search for bridge information
- ✅ How agents work together in a managed system

### Next Steps for the Workshop

1. **Experiment**: Try creating more specialized tools for your bridge domain
2. **Explore**: Look at the geometry agent code to see how it integrates with CAD tools
3. **Design**: Think about what other agents might be useful (structural analysis? cost estimation?)
4. **Build**: Start working on your own specialized agent for the workshop project

### Key Takeaways

- **Tools** are the building blocks - they give agents capabilities
- **CodeAgent** is powerful for complex reasoning and calculations
- **ToolCallingAgent** is predictable and great for production use
- **Managed agents** allow building complex multi-agent systems
- Start simple, test often, and build up complexity gradually

Happy building! 🌉